In [1]:
"""
Fixing the error where didn't import the Dendrites for Spine Overlaying with this table
Was missing from the list so added it and reran so there will be spine data for dendrites


"""

"\nFixing the error where didn't import the Dendrites for Spine Overlaying with this table\nWas missing from the list so added it and reran so there will be spine data for dendrites\n\n\n"

In [2]:
import datajoint as dj
import numpy as np
import time

In [3]:
schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')

Connecting celiib@10.28.0.34:3306


In [4]:
pinky.LeftoverComponentLabelFinal()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,clusters what the clustering parameter was set to,"smoothness what the smoothness parameter was set to, number betwee 0 and 1",n_vertices number of vertices in component,n_triangles number of faces in component,"labeled_vertices indicate which vertices are spine,spine_head,spine_neck otherwise 0","labeled_triangles indicate which faces are spine,spine_head,spine_neck otherwise 0","n_heads totals the number of heads after classification, helps for optimization","n_neck totals the number of heads after classification, helps for optimization","n_spine totals the number of heads after classification, helps for optimization","n_stub totals the number of heads after classification, helps for optimization","n_error totals the number of heads after classification, helps for optimization","used_version whether this component is used in the final labels or not, 0 no, 1 yes"
3,648518346349470171,0.35,Basal,1,12,0.04,4967,9935,=BLOB=,=BLOB=,27,27,4,0,0,1
3,648518346349470171,0.35,Basal,3,12,0.04,3003,6013,=BLOB=,=BLOB=,16,16,0,0,0,1
3,648518346349470171,0.35,Basal,4,12,0.04,1381,2755,=BLOB=,=BLOB=,0,0,0,0,0,1
3,648518346349470171,0.35,Basal,5,12,0.04,895,1782,=BLOB=,=BLOB=,0,0,0,0,0,1
3,648518346349470171,0.35,Basal,6,12,0.04,3140,6251,=BLOB=,=BLOB=,12,12,6,0,0,1
3,648518346349470171,0.35,Basal,8,12,0.04,1094,2184,=BLOB=,=BLOB=,7,7,0,0,0,1
3,648518346349470171,0.35,Basal,9,12,0.04,3197,6364,=BLOB=,=BLOB=,16,16,7,0,0,1


In [5]:
@schema
class LeftoverOverlayedSpineLabel(dj.Computed):
    definition = """
    # Segment labels with Spine labels overlayed (replacing) normal component labels.
    -> pinky.LeftoverCompartmentFinal
    ---
    vertices             : longblob                     
    triangles            : longblob  
    """

In [7]:
#####Need to rerun after 2/20/18 when caught error#####
@schema
class LeftoverOverlayedSpineLabel(dj.Computed):
    definition = LeftoverOverlayedSpineLabel.describe()
    
    def make(self, key):
#         print(key['segment_id'])
#         start = time.time()
        
        #gets the labels from the proofreading
        corrected_labels = (pinky.LeftoverCoarseLabelFinal & key).fetch1()
        
        compartment_key = dict(segmentation=key['segmentation'], segment_id=key['segment_id'], decimation_ratio=0.35)
        #gets all of the components vertex and face indicies bt not the labels
        components = (pinky.LeftoverCompartmentFinal.LeftoverComponentFinal & compartment_key
                      & [dict(compartment_type=compartment) for compartment in ["Apical", "Oblique", "Basal","Dendrite"]]).fetch()
        
        non_existence_count = 0
        component_spine_match = []
        for component in components:
            component_key = dict(compartment_key, compartment_type=component[3], component_index=component[4])
            try:
                #gets the final spine labels for that component
                component_labels = (pinky.LeftoverComponentLabelFinal & component_key).fetch()
                #will get the number of heads 
                n_heads = list(zip(*component_labels))[11]
#                 min_n_heads = np.min(n_heads)
#                 index_supposed_best_fit = np.where(n_heads==min_n_heads)[0][0]#[-1]
#                 component_spine_match.append((component, list(component_labels[index_supposed_best_fit])[8:12]))

                #appends the component data along with the labels to a component spine matching list
                component_spine_match.append((component, list(component_labels[0])[7:11]))
            except:
                non_existence_count += 1
                pass
        print(f"{non_existence_count} components did not have spine data.")
        for match in component_spine_match:
            if not np.array_equal(list(match[0])[5:7], list(match[1])[:2]):
                print(match[0])
                print("Ruh-roh the sizes of the component and component_label array don't match.")
        
        corrected_vertex_labels = corrected_labels['vertices']
        corrected_triangle_labels = corrected_labels['triangles']

        new_complete_vertex_labels = corrected_vertex_labels.copy()
        for match in component_spine_match:
            vertex_indices = match[0][7]
            spine_vertex_labels = match[1][2].astype(np.uint8)
            overlayed_vertex_labels = np.array([corrected_vertex_labels[vertex_indices], spine_vertex_labels]).max(axis=0)
            new_complete_vertex_labels[vertex_indices] = overlayed_vertex_labels

        new_complete_triangle_labels = corrected_triangle_labels.copy()
        for match in component_spine_match:
            triangle_indices = match[0][8]
            spine_triangle_labels = match[1][3].astype(np.uint8)
            overlayed_triangle_labels = np.array([corrected_triangle_labels[triangle_indices], spine_triangle_labels]).max(axis=0)
            new_complete_triangle_labels[triangle_indices] = overlayed_triangle_labels
        
        self.insert1(dict(key,
                          vertices=new_complete_vertex_labels,
                          triangles=new_complete_triangle_labels))
        
#         print(time.time() - start, "\n")

# Segment labels with Spine labels overlayed (replacing) normal component labels.
-> pinky.LeftoverCompartmentFinal
---
vertices             : longblob                     
triangles            : longblob                     



In [10]:
LeftoverOverlayedSpineLabel()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,vertices,triangles
3,648518346341371119,0.35,=BLOB=,=BLOB=
3,648518346349386137,0.35,=BLOB=,=BLOB=
3,648518346349470171,0.35,=BLOB=,=BLOB=
3,648518346349471156,0.35,=BLOB=,=BLOB=
3,648518346349471500,0.35,=BLOB=,=BLOB=
3,648518346349471562,0.35,=BLOB=,=BLOB=
3,648518346349471565,0.35,=BLOB=,=BLOB=


In [9]:
start = time.time()
LeftoverOverlayedSpineLabel.populate()
print(time.time() - start)

3 components did not have spine data.
0 components did not have spine data.
4 components did not have spine data.
1 components did not have spine data.
2 components did not have spine data.
5 components did not have spine data.
3 components did not have spine data.
5 components did not have spine data.
3 components did not have spine data.
1 components did not have spine data.
6 components did not have spine data.
4 components did not have spine data.
1 components did not have spine data.
4 components did not have spine data.
5 components did not have spine data.
3 components did not have spine data.
2 components did not have spine data.
1 components did not have spine data.
1 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
7 components did not have spine data.
2 components did not have spine data.
9 components did not have spine data.
3 components did not have spine data.
0 components did not have spine data.
1 components

In [ ]:
"""
The trimesh method created a different number of vertices than the compartment component method --> so there was a mismatch

Solution: Need to rerun the spine using the Compartment Component solution


Fix is correctly working, the components that are said to not have spine data are those that are too small 
so without size threshold it is 2570 whereas if the size threshold is applied it is 2316

"""